# DEMO de Fine-tuning de OpenAI

Documentación: https://platform.openai.com/docs/guides/fine-tuning

In [ ]:
from openai import OpenAI
# Poned vuestra API KEY de openAI aquí:
openai_api_key = "<OPENAI_API_KEY>"



###   Fine-tuning de modelo gpt-4o-mini





1.   **Preparar los datos de fine-tuning**



In [ ]:
import json
import pandas as pd

finetune_datafile = 'demo_finetuning_data.jsonl' #Fichero con datos de entrenamiento para finetuning (formato jsonl)
data = []
with open(finetune_datafile, 'r') as f:
    for line in f:
        data.append(json.loads(line))
df = pd.DataFrame(data)
df



2.   **Subir archivo de datos a OpenAI**



In [ ]:
from openai import OpenAI
client = OpenAI(api_key=openai_api_key)

file_obj = client.files.create(
  file=open(finetune_datafile, "rb"),
  purpose="fine-tune"
)
print(file_obj)



3.   **Crear proceso de fine-tuning en OpenAI**



In [ ]:
job = client.fine_tuning.jobs.create(
    training_file=file_obj.id,
    model="gpt-4o-mini-2024-07-18"
)
print(job)

4.   **Esperar finalización del proceso de fine-tuning**

In [ ]:
import time
while client.fine_tuning.jobs.retrieve(job.id).status not in ["succeeded", "failed"]:
    #print(client.fine_tuning.jobs.retrieve(job.id).status)
    time.sleep(5)
status = client.fine_tuning.jobs.retrieve(job.id).status
if status== "failed":
    print("Proceso de fine-tuning ha fallado.")
elif status == "succeeded":
    print("Proceso de fine-tuning finalizado con exito.")

In [ ]:
#fine_tuned_model = client.fine_tuning.jobs.retrieve(job.id).fine_tuned_model
fine_tuned_model = "ft:gpt-4o-mini-2024-07-18:ging-upm::Asbwp7rW"

5.   **Realizar inferencia con nuevo modelo afinado**

In [ ]:
completion = client.chat.completions.create(
    model=fine_tuned_model,
    messages=[
        {"role": "system", "content": "Eres un chatbot que clasifica el correo en trabajo, salud, viajes"},
        {"role": "user", "content": "Ven a disfrutar de las playas cálidas de Cancún. ¡Aprovecha ahora la oferta de tiempo limitado!"}
    ]
)

print(completion.choices[0].message.content)

In [ ]:
completion = client.chat.completions.create(
    model=fine_tuned_model,
    messages=[
        {"role": "system", "content": "Eres un chatbot que clasifica el correo en trabajo, salud, viajes"},
        {"role": "user", "content": "No te lo piensas más y mejora tu sonrisa con Vitaldent."}
    ]
)

print(completion.choices[0].message.content)

In [ ]:
completion = client.chat.completions.create(
    model=fine_tuned_model,
    messages=[
        {"role": "system", "content": "Eres un chatbot que clasifica el correo en trabajo, salud, viajes"},
        {"role": "user", "content": "La nueva política interna incorporará nuevas medidas según lo descrito en el RGDP."}
    ]
)

print(completion.choices[0].message.content)